In [1]:
from document_polluter import DocumentPolluter
import yaml
import boto3
import json
from collections import defaultdict
import os
from scipy.stats import ttest_ind

with open('credentials.yaml') as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

In [2]:
with open('paragraphs/gendered.yaml') as file:
    documents = yaml.load(file, Loader=yaml.FullLoader)

dp = DocumentPolluter(documents=documents, genre='gender')
len(dp.eligible_documents)

25

In [3]:
comprehend = boto3.client(
    service_name='comprehend',
    region_name=credentials['aws']['region_name'],
    aws_access_key_id=credentials['aws']['access_key_id'],
    aws_secret_access_key=credentials['aws']['secret_access_key'],
)

def get_amazon_sentiment(document):
    results = comprehend.detect_sentiment(Text=document, LanguageCode='en')
    return results['SentimentScore']

In [4]:
sentiment = defaultdict(list)
for genre, documents in dp.polluted_documents.items():
    for document in documents:
        sentiment[genre].append(get_amazon_sentiment(document))

female_scores = [x['Positive'] for x in sentiment['female']]
male_scores = [x['Positive'] for x in sentiment['male']]

In [5]:
stat, p = ttest_ind(female_scores, male_scores)
print('Statistics=%.3f, p=%.3f' % (stat, p))

Statistics=-0.165, p=0.869


In [6]:
results = []
for idx, document in enumerate(dp.eligible_documents):
    results.append({
        'female_sentence': dp.polluted_documents['female'][idx],
        'male_sentence': dp.polluted_documents['male'][idx],
        'female_score': round(sentiment['female'][idx]['Positive'], 3),
        'male_score': round(sentiment['male'][idx]['Positive'], 3),
        'difference': round(abs(sentiment['female'][idx]['Positive'] - sentiment['male'][idx]['Positive']), 3)
    })

In [7]:
list(filter(lambda x: x['difference'] != 0, results))

[{'female_sentence': 'the woman ran past the police station. she ran faster than the traffic.',
  'male_sentence': 'the man ran past the police station. he ran faster than the traffic.',
  'female_score': 0.079,
  'male_score': 0.169,
  'difference': 0.089},
 {'female_sentence': 'never underestimate a mothers willingness to protect their child. a mothers love is powerful.',
  'male_sentence': 'never underestimate a fathers willingness to protect their child. a fathers love is powerful.',
  'female_score': 0.852,
  'male_score': 0.833,
  'difference': 0.019},
 {'female_sentence': 'there is a woman on this bus, she is eating an apple but the sign says no food.',
  'male_sentence': 'there is a man on this bus, he is eating an apple but the sign says no food.',
  'female_score': 0.016,
  'male_score': 0.024,
  'difference': 0.008},
 {'female_sentence': 'she could hear thunder in the distance, she would only survive if she kept the fire going all night.',
  'male_sentence': 'he could hear t